# SQL100本ノック

In [ ]:
!pip install mysql-connector-python

In [1]:
import os
import mysql.connector

## MySQLと接続出来ることを確認

In [4]:
# 接続パラメータの設定
config = {
  'host': os.environ['MYSQL_IP'], 
  'user': os.environ['MYSQL_USR'],
  'password': os.environ['MYSQL_PW'],
  'database': 'test'
}

# データベース接続
conn = mysql.connector.connect(**config)

# カーソルの作成
cursor = conn.cursor()

# SQLクエリの実行（例）
cursor.execute("SELECT * FROM test")

# データの取得
for row in cursor:
    print(row)


(1, 'yamada')
